In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Load the MNIST dataset from Keras
## Split into training and validation sets

In [4]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

4431872/4422102 [==============================] - 0s 0us/step


Let's split the fashion MNIST training set in two:
* `X_train_A`: all images of all items except for sandals and shirts (classes 5 and 6).
* `X_train_B`: a much smaller training set of just the first 200 images of sandals or shirts.

The validation set and the test set are also split this way, but without restricting the number of images.

We will train a model on set A (classification task with 8 classes), and try to reuse it to tackle set B (binary classification). We hope to transfer a little bit of knowledge from task A to task B, since classes in set A (sneakers, ankle boots, coats, t-shirts, etc.) are somewhat similar to classes in set B (sandals and shirts). However, since we are using `Dense` layers, only patterns that occur at the same location can be reused (in contrast, convolutional layers will transfer much better, since learned patterns can be detected anywhere on the image, as we will see in the CNN chapter).

In [5]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [6]:
tf.random.set_seed(42)
np.random.seed(42)

In [8]:
model_A = keras.models.Sequential()
# first flatten input data
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
# add five dense layers of various neuron amounts
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [9]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [10]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.5803 - accuracy: 0.8143 - val_loss: 0.3851 - val_accuracy: 0.8677
Epoch 2/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.3580 - accuracy: 0.8774 - val_loss: 0.3227 - val_accuracy: 0.8916
Epoch 3/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.3210 - accuracy: 0.8899 - val_loss: 0.2995 - val_accuracy: 0.8994
Epoch 4/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.3016 - accuracy: 0.8969 - val_loss: 0.2868 - val_accuracy: 0.9036
Epoch 5/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.2882 - accuracy: 0.9017 - val_loss: 0.2767 - val_accuracy: 0.9071
Epoch 6/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2780 - accuracy: 0.9039 - val_loss: 0.2716 - val_accuracy: 0.9108
Epoch 7/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.2697 - accuracy: 0.9077 - val_loss: 0.2708 - val_accuracy:

In [11]:
model_A.save("my_model_A.h5")

In [12]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [13]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [14]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 40ms/step - loss: 0.5486 - accuracy: 0.7300 - val_loss: 0.4821 - val_accuracy: 0.8012
Epoch 2/20
7/7 [==============================] - 0s 17ms/step - loss: 0.3941 - accuracy: 0.8550 - val_loss: 0.3780 - val_accuracy: 0.8682
Epoch 3/20
7/7 [==============================] - 0s 16ms/step - loss: 0.3016 - accuracy: 0.9150 - val_loss: 0.3135 - val_accuracy: 0.9087
Epoch 4/20
7/7 [==============================] - 0s 15ms/step - loss: 0.2448 - accuracy: 0.9600 - val_loss: 0.2684 - val_accuracy: 0.9260
Epoch 5/20
7/7 [==============================] - 0s 16ms/step - loss: 0.2043 - accuracy: 0.9750 - val_loss: 0.2376 - val_accuracy: 0.9391
Epoch 6/20
7/7 [==============================] - 0s 17ms/step - loss: 0.1761 - accuracy: 0.9800 - val_loss: 0.2146 - val_accuracy: 0.9483
Epoch 7/20
7/7 [==============================] - 0s 15ms/step - loss: 0.1553 - accuracy: 0.9800 - val_loss: 0.1972 - val_accuracy: 0.9533
Epoch 8/20
7/7 [===========

In [15]:
model_B.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_12 (Dense)            (None, 300)               235500    
                                                                 
 dense_13 (Dense)            (None, 100)               30100     
                                                                 
 dense_14 (Dense)            (None, 50)                5050      
                                                                 
 dense_15 (Dense)            (None, 50)                2550      
                                                                 
 dense_16 (Dense)            (None, 50)                2550      
                                                                 
 dense_17 (Dense)            (None, 1)                

In [16]:
model_A = keras.models.load_model("my_model_A.h5")

# removing last layer of model A
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
# adding new layer
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

Because of how Python functions work, we actually need to "clone" Model A before we reuse its layers. 

In [17]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

Now, we need to "freeze" the layers we are reusing, by setting their "trainable" attribute to False and compiling the model:

In [18]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

In [19]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 40ms/step - loss: 1.1684 - accuracy: 0.3350 - val_loss: 1.0884 - val_accuracy: 0.3813
Epoch 2/4
7/7 [==============================] - 0s 14ms/step - loss: 1.0449 - accuracy: 0.3700 - val_loss: 0.9753 - val_accuracy: 0.4219
Epoch 3/4
7/7 [==============================] - 0s 14ms/step - loss: 0.9320 - accuracy: 0.4400 - val_loss: 0.8765 - val_accuracy: 0.4746
Epoch 4/4
7/7 [==============================] - 0s 16ms/step - loss: 0.8338 - accuracy: 0.5000 - val_loss: 0.7953 - val_accuracy: 0.5304
Epoch 1/16
7/7 [==============================] - 1s 40ms/step - loss: 0.6108 - accuracy: 0.6450 - val_loss: 0.4404 - val_accuracy: 0.8032
Epoch 2/16
7/7 [==============================] - 0s 17ms/step - loss: 0.3438 - accuracy: 0.8850 - val_loss: 0.2912 - val_accuracy: 0.9209
Epoch 3/16
7/7 [==============================] - 0s 17ms/step - loss: 0.2290 - accuracy: 0.9650 - val_loss: 0.2203 - val_accuracy: 0.9523
Epoch 4/16
7/7 [===============

In [20]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.0931 - accuracy: 0.9835


[0.09306484460830688, 0.9835000038146973]

In [21]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0666 - accuracy: 0.9910


[0.06663096696138382, 0.9909999966621399]

Great! We got quite a bit of transfer: the error rate dropped by a factor of 4.9

In [22]:
(100 - 97.05) / (100 - 99.40)

4.916666666666718

# Conclusions
Wow! It looks like transfer learning is great! Well it is, but unfortunately, some cheating has gone one here. The author already knows which form of tranfer learning will work best. Therefore, we don't know without some trial-and-error which layers we ought to keep.